In [ ]:
# 2025-12-06 imported those with GNDs
from csae_pyutils import gsheet_to_df
from django.core.exceptions import ObjectDoesNotExist

from tqdm import tqdm
import pandas as pd
from normdata.utils import import_from_normdata

In [ ]:
df = gsheet_to_df("1-oRC9J8VjHvt2jym3AdZBWS3klZv3T_uT8rkUZHkb6I")

In [ ]:
df.head()

In [ ]:
col, _ = Collection.objects.get_or_create(name="Briefedition Wedekind")
domain = "wedekind"
col_type, _ = CollectionType.objects.get_or_create(name="Projekt")
col.description = 'Frank Wedekinds Korrespondenz digital. <a href="https://briefedition.wedekind.fernuni-hagen.de">https://briefedition.wedekind.fernuni-hagen.de<a>'
col.collection_type = col_type
col.save()

In [ ]:
# process those with GND first
data = {}
for i, row in df.iterrows():
    if row["gender"] in ["männlich", "weiblich"]:
        data[row["URI"]] = f"https://d-nb.info/gnd/{row['gnd']}"
cleaned_data = {}
for key, value in data.items():
    if not value.endswith("gnd/nan"):
        cleaned_data[key] = value

In [ ]:
broken_gnd = []
pmb_uris = []
for key, value in tqdm(cleaned_data.items()):
    entity = import_from_normdata(value, "person")
    if entity:
        pmb_uri, _ = Uri.objects.get_or_create(uri=key, domain=domain)
        pmb_uri.entity = entity
        pmb_uri.save()
        entity.collection.add(col)
        pmb_uris.append([key, value])
    else:
        broken_gnd.append([key, value])

In [ ]:
for x in broken_gnd:
    print(x)